<!-- TABS -->
# Transfer learning

<!-- TABS -->
## Connect to superduper

In [1]:
from superduper import superduper

db = superduper('mongomock:///test_db')

2024-Aug-30 21:34:07.16| INFO     | Duncans-MBP.fritz.box| superduper.misc.plugins:13   | Loading plugin: mongodb
2024-Aug-30 21:34:07.29| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:103  | Building Data Layer
2024-Aug-30 21:34:07.29| INFO     | Duncans-MBP.fritz.box| superduper.base.build:171  | Configuration: 
 +---------------+----------------------+
| Configuration |        Value         |
+---------------+----------------------+
|  Data Backend | mongomock:///test_db |
+---------------+----------------------+


<!-- TABS -->
## Get useful sample data

In [2]:
# <tab: Text-Classification>
!curl -O https://superduperdb-public-demo.s3.amazonaws.com/text_classification.json
import json

with open("text_classification.json", "r") as f:
    data = json.load(f)
num_classes = 2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1298k  100 1298k    0     0  1127k      0  0:00:01  0:00:01 --:--:-- 1130k


In [ ]:
# <tab: Image-Classification>
!curl -O https://superduperdb-public-demo.s3.amazonaws.com/images_classification.zip && unzip images_classification.zip
import json
from PIL import Image

with open('images/images.json', 'r') as f:
    data = json.load(f)
    
data = [{'x': Image.open(d['image_path']), 'y': d['label']} for d in data]
num_classes = 2

After obtaining the data, we insert it into the database.

In [3]:
# <tab: Text-Classification>
datas = [{'txt': d['x'], 'label': d['y']} for d in data]

In [ ]:
# <tab: Image-Classification>
datas = [{'image': d['x'], 'label': d['y']} for d in data]

<!-- TABS -->
## Insert simple data

After turning on auto_schema, we can directly insert data, and superduper will automatically analyze the data type, and match the construction of the table and datatype.

In [4]:
from superduper import Document

table_or_collection = db['docs']

ids = db.execute(table_or_collection.insert([Document(data) for data in datas]))
select = table_or_collection.select()

2024-Aug-30 21:34:13.87| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:363  | Table docs does not exist, auto creating...
2024-Aug-30 21:34:13.87| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:369  | Creating table docs with schema {('label', 'int'), ('_fold', 'str'), ('txt', 'str')}
2024-Aug-30 21:34:13.88| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:415  | Leaf str already exists
2024-Aug-30 21:34:13.98| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:344  | Inserted 1000 documents into docs


<!-- TABS -->
## Compute features

In [5]:
# <tab: Text>
key = 'txt'
import sentence_transformers
from superduper import vector, Listener
from superduper_sentence_transformers import SentenceTransformer

superdupermodel = SentenceTransformer(
    identifier="embedding",
    object=sentence_transformers.SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2"),
    postprocess=lambda x: x.tolist(),
)

jobs, listener = db.apply(
    Listener(
        model=superdupermodel,
        select=select,
        key=key,
        identifier="features"
    )
)

/Users/dodo/.pyenv/versions/3.11.7/envs/superduper-3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/dodo/.pyenv/versions/3.11.7/envs/superduper-3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


2024-Aug-30 21:34:23.96| INFO     | Duncans-MBP.fritz.box| superduper.components.listener:94   | Requesting listener setup on CDC service
2024-Aug-30 21:34:23.96| INFO     | Duncans-MBP.fritz.box| superduper.components.listener:104  | Skipping listener setup on CDC service since no URI is set
2024-Aug-30 21:34:23.96| INFO     | Duncans-MBP.fritz.box| superduper.jobs.queue:210  | Running jobs for listener::features
2024-Aug-30 21:34:23.96| INFO     | Duncans-MBP.fritz.box| superduper.backends.local.compute:67   | Submitting job. function:<function method_job at 0x1121a74c0>
2024-Aug-30 21:34:23.99| INFO     | Duncans-MBP.fritz.box| superduper.components.model:720  | Requesting prediction in db - [embedding] with predict_id features



/Users/dodo/.pyenv/versions/3.11.7/envs/superduper-3.11/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io

2024-Aug-30 21:34:28.66| INFO     | Duncans-MBP.fritz.box| superduper.components.model:853  | Adding 1000 model outputs to `db`
2024-Aug-30 21:34:29.94| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:344  | Inserted 1000 documents into _outputs__features
2024-Aug-30 21:34:29.94| SUCCESS  | Duncans-MBP.fritz.box| superduper.backends.local.compute:73   | Job submitted on <superduper.backends.local.compute.LocalComputeBackend object at 0x2a63a6950>.  function:<function method_job at 0x1121a74c0> future:2638f7c5-655d-47e6-ba13-a214afd3dd3c


In [ ]:
# <tab: Image>
key = 'image'
import torchvision.models as models
from torchvision import transforms
from superduper_torch import TorchModel
from superduper import Listener
from PIL import Image

class TorchVisionEmbedding:
    def __init__(self):
        # Load the pre-trained ResNet-18 model
        self.resnet = models.resnet18(pretrained=True)
        
        # Set the model to evaluation mode
        self.resnet.eval()
        
    def preprocess(self, image):
        # Preprocess the image
        preprocess = preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        tensor_image = preprocess(image)
        return tensor_image
        
model = TorchVisionEmbedding()
superdupermodel = TorchModel(identifier='my-vision-model-torch', object=model.resnet, preprocess=model.preprocess, postprocess=lambda x: x.numpy().tolist())

jobs, listener = db.apply(
    Listener(
        model=superdupermodel,
        select=select,
        key=key,
        identifier="features"
    )
)

## Choose features key from feature listener

In [6]:
input_key = listener.outputs
training_select = select.outputs(listener.predict_id)

We can find the calculated feature data from the database.

In [7]:
feature = list(training_select.limit(1).execute())[0][input_key]
feature_size = len(feature)

<!-- TABS -->
## Build and train classifier

In [8]:
# <tab: Scikit-Learn>
from superduper_sklearn import Estimator, SklearnTrainer
from sklearn.svm import SVC

model = Estimator(
    identifier="my-model",
    object=SVC(),
    trainer=SklearnTrainer(
        "my-trainer",
        key=(input_key, "label"),
        select=training_select,
    ),
)

In [ ]:
# <tab: Torch>
import torch
from torch import nn
from superduper_torch.model import TorchModel
from superduper_torch.training import TorchTrainer
from torch.nn.functional import cross_entropy


class SimpleModel(nn.Module):
    def __init__(self, input_size=16, hidden_size=32, num_classes=3):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

preprocess = lambda x: torch.tensor(x)

# Postprocess function for the model output    
def postprocess(x):
    return int(x.topk(1)[1].item())

def data_transform(features, label):
    return torch.tensor(features), label

# Create a Logistic Regression model
# feature_length is the input feature size
model = SimpleModel(feature_size, num_classes=num_classes)
model = TorchModel(
    identifier='my-model',
    object=model,         
    preprocess=preprocess,
    postprocess=postprocess,
    trainer=TorchTrainer(
        key=(input_key, 'label'),
        identifier='my_trainer',
        objective=cross_entropy,
        loader_kwargs={'batch_size': 10},
        max_iterations=1000,
        validation_interval=100,
        select=select,
        transform=data_transform,
    ),
)

Define a validation for evaluating the effect after training.

In [9]:
from superduper import Dataset, Metric, Validation


def acc(x, y):
    return sum([xx == yy for xx, yy in zip(x, y)]) / len(x)


accuracy = Metric(identifier="acc", object=acc)
validation = Validation(
    "transfer_learning_performance",
    key=(input_key, "label"),
    datasets=[
        Dataset(identifier="my-valid", select=training_select.add_fold('valid'))
    ],
    metrics=[accuracy],
)
model.validation = validation

If we execute the apply function, then the model will be added to the database, and because the model has a Trainer, it will perform training tasks.

In [10]:
db.apply(model)

2024-Aug-30 21:34:35.43| WARNING  | Duncans-MBP.fritz.box| superduper.backends.local.artifacts:85   | File /tmp/test_db/11f4c3e68e30071c92163ece040096ea80e23755 already exists


([],
 Estimator(trainer=SklearnTrainer(identifier='my-trainer', uuid='4ed22cfa3a9c4175ae89b7511eda1afb', upstream=None, plugins=None, cache=False, key=('_outputs__features', 'label'), select=docs.select().outputs("features"), transform=None, metric_values={}, signature='*args', data_prefetch=False, prefetch_size=1000, prefetch_factor=100, in_memory=True, compute_kwargs={}, fit_params={}, predict_params={}, y_preprocess=None), identifier='my-model', uuid='1e6b4f4000ac4cb8b5123e2cc944a071', upstream=None, plugins=None, cache=False, signature='singleton', datatype=None, output_schema=None, flatten=False, model_update_kwargs={}, predict_kwargs={}, compute_kwargs={}, validation=Validation(identifier='transfer_learning_performance', uuid='b01186bf87cb49b0a6dcf86f53a660c5', upstream=None, plugins=None, cache=False, metrics=[Metric(identifier='acc', uuid='e83f664b7f6947cfb67947e472b7d467', upstream=None, plugins=None, cache=False, object=<function acc at 0x2d41cae80>)], key=('_outputs__feature

In [11]:
model.encode()

{'_base': '?my-model',
 '_builds': {'docs-select-outputs-features': {'_path': 'superduper_mongodb.query.parse_query',
   'documents': [],
   'query': 'docs.select().outputs("features")'},
  'my-trainer': {'_path': 'superduper_sklearn.model.SklearnTrainer',
   'uuid': '4ed22cfa3a9c4175ae89b7511eda1afb',
   'upstream': None,
   'plugins': None,
   'cache': False,
   'key': ('_outputs__features', 'label'),
   'select': '?docs-select-outputs-features',
   'transform': None,
   'metric_values': {},
   'signature': '*args',
   'data_prefetch': False,
   'prefetch_size': 1000,
   'prefetch_factor': 100,
   'in_memory': True,
   'compute_kwargs': {},
   'fit_params': {},
   'predict_params': {},
   'y_preprocess': None,
   'type_id': 'trainer',
   'version': 0,
   'hidden': False},
  'dill': {'_path': 'superduper.components.datatype.get_serializer',
   'method': 'dill',
   'encodable': 'artifact',
   'type_id': 'datatype',
   'version': None,
   'uuid': '2e98be744c274ffabc472e74bcdb8b13'},
  '

Get the training metrics

In [ ]:
model = db.load('model', model.identifier)
model.metric_values

In [12]:
from superduper import Template

t = Template('transfer-learner', template=model, substitutions={'docs': 'table'})

In [13]:
t.export('.')